# Jupyter Notebook: 2D Spectrum Processing

This notebook demonstrates how to:
1. Read complex data from files and assemble non-rephasing and rephasing signals.
2. Perform a 2D Fast Fourier Transform (FFT) on these signals.
3. Apply interpolation (both bilinear and then optional high-resolution interpolation).
4. Normalize and visualize the resulting 2D spectrum.

Make sure the file paths are correctly set before running.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy.interpolate import griddata, RegularGridInterpolator
from scipy.fft import fft2, fftshift, fftfreq
from scipy.ndimage import gaussian_filter

## 1. Define Parameters and Initialize Arrays

Here, we set:

- The grid sizes (`size`, `size1`, `size2`), which might be relevant for the expected data dimensions.

- Time-related parameters (`time`, `t_step`, `tau_step`, etc.).

- File naming parameters.

- Empty arrays to store the non-rephasing and rephasing data

In [ ]:
# Time parameters
time = 1000
t_size = 0      # size of detection time data at fixed coherent time.
t_step = 10     # detection time step.
tau_size = 0    # size of coherent time data at fixed detection time.
tau_step = 10   # coherent time step.
T = 0           # Population time.

# Tau range
tau_start = -600
tau_end = 600

# Grid sizes (modify as needed)
size1 = 1000                                                # Storge space size of detection time data at fixed coherent time with pedding zeros.
size2 = int((tau_end - tau_start) / tau_step / 2) + 1       # Storge space size of coherent time data at fixed detection time.

## 2. Load and Assemble Non-Rephasing Data

We loop over the nagetive **tau** range for non-rephasing data:

1. Select the file name based on chosen tau.

2. Read each file as pairs of real and imaginary parts.

3. Convert to complex values and store them in `non_rephaasing_data`.

4. Update the dimensions of `t_size` and `tau_size`.

In [ ]:
# Non-rephasing time windows
non_rephaasing_tau_start = tau_start
non_rephaasing_tau_end = 0

# Arrays to store non-rephasing data
non_rephaasing_data = np.zeros((size1, size2), dtype=np.complex128)

for i, tau in enumerate(np.arange(non_rephaasing_tau_start, non_rephaasing_tau_end + 1, tau_step)):
    filename = f"../2d_output/out_{tau}_{T}.out"
    
    with open(filename, 'r') as file:
        # Load file content as complex numbers
        complex_numbers = np.loadtxt(filename, dtype=complex, 
                                     converters={0: lambda x: complex(float(x), 0), 
                                                 1: lambda x: complex(0, float(x))})
        
        # Combine real and imaginary parts into complex
        d = np.array([complex(r.real, i.imag) for r, i in complex_numbers])
        
        # Multiply by 1j if needed (as per your original logic)
        d = d * 1j
        
        # Fill non_rephaasing_data array
        for j in range(size1):
            if j < len(d):
                non_rephaasing_data[j][i] = d[j]
            else:
                non_rephaasing_data[j][i] = 0

    if i == 0:
        t_size = len(d)
    tau_size += 1

## 3. Load and Assemble Rephasing Data

Similar approach as for non-rephasing, but for the range from 0 to `tau_end`.

In [ ]:
rephaasing_data = np.zeros((size1, size2), dtype=np.complex128)
rephasing_tau_start = 0
rephasing_tau_end = tau_end

for i, tau in enumerate(np.arange(rephasing_tau_start, rephasing_tau_end + 1, tau_step)):
    filename = f"../2d_output/out_{tau}_{T}.out"
    
    with open(filename, 'r') as file:
        # Load file content as complex numbers
        complex_numbers = np.loadtxt(filename, dtype=complex, 
                                     converters={0: lambda x: complex(float(x), 0), 
                                                 1: lambda x: complex(0, float(x))})
        
        # Combine real and imaginary parts
        d = np.array([complex(r.real, i.imag) for r, i in complex_numbers])
        
        # Multiply by 1j if needed
        d = d * 1j
        
        # Fill rephaasing_data array
        for j in range(size1):
            if j < len(d):
                rephaasing_data[j][i] = d[j]
            else:
                rephaasing_data[j][i] = 0

## 4. Perform 2D FFT and Combine

1. We compute the 2D Fourier transform of both non-rephasing and rephasing data.

2. We use `fftshift` to center the zero-frequency component.

3. We take the real part of the result.

4. Combine them into a single `fft_data`.

In [ ]:
fft_non_rephaasing_data = fftshift(fft2(non_rephaasing_data)).real
fft_rephaasing_data = fftshift(fft2(rephaasing_data)).real

fft_data = fft_rephaasing_data + fft_non_rephaasing_data

## 5. (Optional) Apply Gaussian Filter

If you want to smooth the data, you can use `gaussian_filter`. Adjust `sigma` values as needed.

In [ ]:
fft_data = gaussian_filter(fft_data, sigma=(10, 0.6))  # Uncomment if needed

## 6. Interpolate the FFT Data

1. We set an interpolation factor (`interp_factor`).

2. Create new grids along each axis.

3. Use `RegularGridInterpolator` for bilinear interpolation.

4. Update `fft_data` to the interpolated version.

In [ ]:
interp_factor = 2
new_size1 = fft_data.shape[0] * interp_factor
new_size2 = fft_data.shape[1] * interp_factor

# Original grid
x = np.arange(fft_data.shape[0])
y = np.arange(fft_data.shape[1])

# New grid for interpolation
new_x = np.linspace(0, fft_data.shape[0] - 1, new_size1)
new_y = np.linspace(0, fft_data.shape[1] - 1, new_size2)

# Setup the interpolator
interpolator = RegularGridInterpolator((x, y), fft_data)

# Generate the new points (mesh)
new_grid = np.meshgrid(new_x, new_y, indexing='ij')
new_points = np.array([new_grid[0].ravel(), new_grid[1].ravel()]).T

# Interpolate
interpolated_data = interpolator(new_points).reshape(new_size1, new_size2)
fft_data = interpolated_data


## 7. Prepare Frequency Axes

We use `fftshift` and `fftfreq` to compute frequency arrays for `t` and `tau` axes. The factor `5308 * 2 * np.pi` seems to be a conversion to a specific unit (e.g., cm^{-1}).


In [ ]:
t = np.linspace(0, time + (new_size1 - t_size) * t_step, new_size1)
tau = np.linspace(0, tau_end + (new_size2 - tau_size) * tau_step, new_size2)

t = 5308 * 2 * np.pi * fftshift(fftfreq(len(t), t[1] - t[0]))
tau = 5308 * 2 * np.pi * fftshift(fftfreq(len(tau), tau[1] - tau[0]))


## 8. High-Resolution Interpolation (Optional)

We do a further interpolation onto an even finer grid (`high_res_factor`) using `scipy.interpolate.griddata` (cubic interpolation). This step is primarily for smooth contour plotting.


In [ ]:
high_res_factor = 8
tau_high_res = np.linspace(tau.min(), tau.max(), size2 * high_res_factor)
t_high_res = np.linspace(t.min(), t.max(), size1 * high_res_factor)
X_high_res, Y_high_res = np.meshgrid(tau_high_res, t_high_res)

X, Y = np.meshgrid(tau, t)
points = np.array([X.flatten(), Y.flatten()]).T
values = fft_data.flatten()

fft_data_high_res = griddata(points, values, (X_high_res, Y_high_res), method='cubic')
fft_data = fft_data_high_res


## 9. Normalize and Plot

1. Normalize the data by its absolute maximum (`fft_data / max_abs_value`).

2. Create a contour plot and a color map (`imshow`).

3. Optionally limit the plotted range in `xlim` and `ylim`.

4. Uncomment lines to add colorbar, axis labels, ticks, etc., based on your preference.


In [ ]:
# Normalize
max_abs_val = np.max(np.abs(fft_data))
fft_data = fft_data / max_abs_val

# Plot settings
fontsize = 30
labelpad = 12
labelsize = 16

plt.figure(figsize=(10, 8.5))

# Contour levels
lv = np.linspace(np.min(fft_data_high_res), np.max(fft_data_high_res), 12)
norm = Normalize(vmin=-1.0, vmax=1.0)

# Draw contour lines
plt.contour(X_high_res, Y_high_res[::-1], fft_data_high_res, colors=['#000', '#000'], 
            levels=lv, norm=norm, linewidths=1)

# Main image plot
img = plt.imshow(
    fft_data,
    extent=(tau.min(), tau.max(), t.min(), t.max()),
    cmap='jet',
    norm=norm
)

# Optional colorbar
plt.colorbar(img, ticks=[-1.0, -0.5, 0, 0.5, 1.0])

# Axis labels
plt.xlabel(r"$\omega_{\tau} (cm^{-1})$", fontsize=fontsize, labelpad=labelpad)
plt.ylabel(r"$\omega_{t} (cm^{-1})$", fontsize=fontsize, labelpad=labelpad)

# Control ticks and range
plt.tick_params(axis='both', which='major', labelsize=labelsize)
plt.xlim(0, 900)
plt.ylim(0, 900)

# Optional title and saving
# plt.title(f'Pulse width = 100 fs')
# plt.savefig(f"T{T}.eps", format='eps', dpi=300)

plt.show()


## 10. Version Information

Below we print out the versions of Python and the main libraries used in this notebook for reproducibility purposes:

In [1]:
import sys
import numpy as np
import scipy
import matplotlib

print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Matplotlib version:", matplotlib.__version__)

Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
NumPy version: 1.24.3
SciPy version: 1.11.1
Matplotlib version: 3.7.1
